In [5]:
import os, glob
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
from zipfile import ZipFile
import pdb

In [12]:
# get data location
#base_path = "/Users/nikkibytes/Documents/niblunc/BCP/BCP_DATA"#"/Users/nikkibytes/Documents/BCP_DATA"
basepath = "/Users/gracer/Google Drive/BCP/data"
#text4_files=glob.glob(os.path.join(base_path, "set*/*/*/*04.txt"))
#text9_files=glob.glob(os.path.join(base_path, "set*/*/*/*09.txt"))

# Read in the data from the zip files

In [13]:
list_of_files = {}
for (dirpath, dirnames, filenames) in os.walk(base_path):
    for filename in filenames:
        if filename.endswith('.zip'):
            #print(filename)
            tmppath=os.sep.join([dirpath, filename])
            #print(tmppath)
            with ZipFile(tmppath, 'r') as zipObj:
               # Get a list of all archived file names from the zip
               listOfFileNames = zipObj.namelist()
               # Iterate over the file names
               for fileName in listOfFileNames:
                   # Check filename endswith csv
                    if fileName.endswith('04.txt'):
                                           # Extract a single file from zip
                        zipObj.extract(fileName, os.path.join(basepath,'temp_txt'))
                    if fileName.endswith('09.txt'):
                                           # Extract a single file from zip
                        zipObj.extract(fileName, os.path.join(basepath,'temp_txt'))
infile = os.path.join(basepath,'temp_txt')

# Dictionary of the files we want

In [24]:
file_dict = {"set_04": {"mom":{}, "child": {}}, "set_09": {"mom": {}, "child": {}}}
file_dict['set_04']["mom"]["files"] = [x for x in glob.glob(os.path.join(infile,'*04.txt')) if "Mom" in x]
file_dict['set_09']["mom"]["files"] = [x for x in glob.glob(os.path.join(infile,'*09.txt')) if "Mom" in x]
file_dict['set_04']["child"]["files"] = [x for x in glob.glob(os.path.join(infile,'*04.txt')) if "Chil" in x]
file_dict['set_09']["child"]["files"] = [x for x in glob.glob(os.path.join(infile,'*09.txt')) if "Chil" in x]


# Concat

In [44]:
temp_list = []            
for file in file_dict["set_04"]["mom"]["files"]:
    temp_df =  pd.read_csv(file, sep="\t", encoding='latin1')
    temp_df=temp_df.drop([0])
    for val in temp_df["Participant ID"]:
        _id = str(val).lstrip("0").split("_")[0]
        temp_df.replace(val, _id, inplace=True)
    #print(updated_df["Participant ID"])
    temp_list.append(temp_df)
    #print(temp_df.shape)
dfm4_original = pd.concat(temp_list, ignore_index=True)
print("Final dataframe size: ", dfm4_original.shape)

dfm4_original.head()
dfm4_orignal = dfm4_original.sort_values(by="Participant ID")
dfm4_original.head()

Final dataframe size:  (360, 226)


,Project Abbreviation,Participant ID,Date of Intake,Project Name,Record Type,Participant Name,Gender,Date of Birth,Date of Entry,Day of Intake,...,Data Field 1 Descriptor,Data Field 1 Response,Data Field 2 Descriptor,Data Field 2 Response,Data Field 3 Descriptor,Data Field 3 Response,Data Field 4 Descriptor,Data Field 4 Response,Data Field 5 Descriptor,Data Field 5 Response
0,BCPMomMSTR,1995,06/06/2017,"Baby ConnectoMe Project, Mom MASTER, Q4 2017",1,NaN,3,NaN,12/14/2017,2,...,Data Field 1,NaN,Data Field 2,NaN,Data Field 3,NaN,Data Field 4,NaN,Data Field 5,NaN
1,BCPMomMSTR,11370,05/30/2017,"Baby ConnectoMe Project, Mom MASTER, Q4 2017",1,NaN,3,NaN,11/21/2017,2,...,Data Field 1,NaN,Data Field 2,NaN,Data Field 3,NaN,Data Field 4,NaN,Data Field 5,NaN
2,BCPMomMSTR,11370,08/23/2017,"Baby ConnectoMe Project, Mom MASTER, Q4 2017",1,NaN,3,NaN,11/21/2017,3,...,Data Field 1,NaN,Data Field 2,NaN,Data Field 3,NaN,Data Field 4,NaN,Data Field 5,NaN
3,BCPMomMSTR,17541,05/30/2017,"Baby ConnectoMe Project, Mom MASTER, Q4 2017",1,NaN,3,NaN,01/02/2018,2,...,Data Field 1,NaN,Data Field 2,NaN,Data Field 3,NaN,Data Field 4,NaN,Data Field 5,NaN
4,BCPMomMSTR,20784,12/13/2016,"Baby ConnectoMe Project, Mom MASTER, Q4 2017",1,NaN,3,NaN,12/14/2017,2,...,Data Field 1,NaN,Data Field 2,NaN,Data Field 3,NaN,Data Field 4,NaN,Data Field 5,NaN


In [45]:
temp_list = []            
for file in file_dict["set_09"]["mom"]["files"]:
    temp_df = pd.read_csv(file,encoding='latin1', sep="\t")
#     print("Loading file {}...........Dataframe size: {}".format(file.split("/")[-1], temp_df.shape))
    temp_df=temp_df.drop([0])
    for val in temp_df["Participant ID"]:
        _id = str(val).strip("0").strip(".").split("_")[0]
        #print(_id)
        temp_df.replace(val, _id, inplace=True)
    #print(updated_df["Participant ID"])
    temp_list.append(temp_df)
dfm9_original = pd.concat(temp_list, ignore_index=True)
dfm9_orignal = dfm9_original.sort_values(by="Participant ID")
print("Final dataframe size: ", dfm9_original.shape)
dfm9_original.head()


Final dataframe size:  (368, 171)


,Project Abbreviation,Participant ID,Date of Intake,FRU0100,FRU0200,FRU0300,FRU0400,FRU0500,FRU0600,FRU0700,...,MSC0300,MSC0400,MSC0500,MSC0600,MSC0700,MSC0800,MSC0900,MSC1000,GRW1300,GRS1300
0,BCPMomAP,11370,11/29/2017,1.500,0,0,0.500,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,BCPMomAP,94794,12/03/2017,0.012,0,0,0,0.354,0,0,...,0,0,0.360,0,0,0,0,0,0,0
2,BCPMomAP,116769,12/19/2017,0,0,0,0,0,0,0,...,0.907,0,0,0,0,0,0,0,0,0
3,BCPMomAP,155866,02/19/2018,0,0,0,0.800,0.907,0,0,...,0,0,0,0,0,0,0,0,0,0
4,BCPMomAP,156965,11/26/2017,0,0,0,0,0,0,0,...,0.250,0.630,0.117,0,0,0,0,0,0,0


# Saving

In [48]:
concat_filepath = os.path.join(basepath,'Concat','Mom_BCP_dataset4.txt')
dfm4_original.to_csv(concat_filepath, index=False, sep="\t", header=True)

In [49]:
concat_filepath = os.path.join(basepath,'Concat','Mom_BCP_dataset9.txt')
dfm9_original.to_csv(concat_filepath, index=False, sep="\t", header=True)

In [91]:
df=dfm4_original

# Remove stupid fucking commas

In [103]:
for col in dfm4_original:
    if dfm4_original[col].dtype == np.object_:
        dfm4_original[col] = (dfm4_original[col].replace(',','.', regex=True))
        

# Groups

In [125]:
dfm9_original['hei_totveg']

0       3.712
1       3.063
2       2.620
3       3.183
4       5.815
5       3.000
6       2.955
7       0.906
8       3.349
9       4.256
10      1.944
11      0.000
12      1.341
13      0.718
14      4.451
15      3.821
16      7.583
17      5.016
18      0.920
19      1.305
20      5.934
21      3.714
22      6.638
23      2.144
24      7.633
25      0.683
26     14.502
27      0.625
28      3.327
29      2.619
        ...  
338     8.157
339     2.112
340     0.255
341     2.007
342     2.214
343     4.390
344     1.333
345     0.125
346     3.613
347     0.803
348     3.991
349     9.329
350     3.969
351     1.473
352     4.705
353     3.562
354     1.500
355     3.453
356     7.090
357     5.487
358     2.500
359     1.303
360     2.011
361     0.478
362     5.363
363     0.007
364     1.903
365     3.073
366     7.074
367     6.718
Name: hei_totveg, Length: 368, dtype: float64

In [222]:
hei_dict={'hei_totveg':
          ['VEG0100','VEG0200','VEG0300','VEG0400','VEG0800','VEG0450','VEG0700','VEG0600','VEG0900','VEG0500'],
          'hei_greensbeans':
          ['VEG0100','VEG0700'],
          'hei_totfruit':
          ['FRU0100','FRU0200','FRU0300','FRU0400','FRU0500','FRU0600','FRU0700'],
          'hei_wholefruit':
          ['FRU0300','FRU0400','FRU0500','FRU0600','FRU0700'],
          'hei_wholegrains':
          ['Whole Grains (ounce equivalents)'],
          'hei_dairy':
          ['DMF0100','DMR0100','DML0100','DMN0100','DMF0200','DMR0200','DML0200',
                       'DML0300','DML0400','DCF0100','DCR0100','DCL0100','DCN0100','DYF0100',
                       'DYR0100','DYL0100','DYF0200','DYR0200','DYL0200','DYN0100',
                       'DOT0300','DOT0400','DOT0500','DOT0600','DOT0100'],
          'hei_totproteins':
          ['MRF0100','MRL0100','MRF0200','MRL0200','MRF0300','MRL0300','MRF0400',
                             'MRL0400','MCF0200','MCL0200','MRF0500','MPF0100','MPL0100','MPF0200',
                             'MFF0100','MFL0100','MFF0200','MSL0100',
                             'MSF0100','MCF0100','MCL0100','MOF0100','MOF0200','MOF0300','MOF0400','MOF0500',
                             'MOF0600','MOF0700','VEG0700'],
          'hei_seafoodplantprot':
          ['MFF0100','MFL0100','MFF0200','MSL0100','MSF0100','MOF0500','MOF0600','MOF0700','VEG0700'],
          'hei_sodium':
          ['Sodium (mg)'],
          'hei_refinedgrains':
          ['Refined Grains (ounce equivalents)'],
          'hei_addedsugars':
          ['Added Sugars (by Total Sugars) (g)'], 
          'ripctsfa': ['% Calories from SFA','Energy (kcal)'],
         'energy':
         ['Energy (kcal)'],
         'fats':
         ['Total Polyunsaturated Fatty Acids (PUFA) (g)','Total Monounsaturated Fatty Acids (MUFA) (g)',
         'Total Saturated Fatty Acids (SFA) (g)']}

In [ ]:
dfm4_original['Total Saturated Fatty Acids (SFA) (g)']

In [189]:
hei_dict.keys()

dict_keys(['hei_totveg', 'hei_greensbeans', 'hei_totfruit', 'hei_wholefruit', 'hei_wholegrains', 'hei_dairy', 'hei_totproteins', 'hei_seafoodplantprot', 'hei_sodium', 'hei_refinedgrains', 'hei_addedsugars', 'ripctsfa', 'energy'])

In [223]:
for key, value in hei_dict.items():
    print(key)
    if key in ['hei_totveg','hei_greensbeans','hei_totfruit', 'hei_wholefruit']:
        x=value
        dfm9_original[key] = dfm9_original[x].astype('float').sum(axis=1)
        dfm9_original[key] = dfm9_original[key]/2
    if key in ['hei_wholegrains','hei_refinedgrains']:
        x=value
        dfm4_original[key] = dfm4_original[x].astype('float')
    if key in ['hei_dairy']:
        x=value[:-1]
        tmp= dfm9_original[x].astype('float').sum(axis=1)
        y=value[-1]
        if y == 'DOT0100':
            tmp2=dfm9_original[y].astype('float')/3
            dfm9_original[key]=tmp+tmp2
        else:
            print('NO DAIRY MISSING DOT0100, needs to be last in list')
    if key in ['hei_totproteins']:
        x=value[:-1]
        tmp= dfm9_original[x].astype('float').sum(axis=1)
        y=value[-1]
        if y == 'VEG0700':
            tmp2=dfm9_original[y].astype('float')*2
            dfm9_original[key]=tmp+tmp2
        else:
            print('NO TOTAL PROTEIN MISSING VEG0700, needs to be last in list')
    if key in ['hei_seafoodplantprot']:
        x=value[:-1]
        tmp= dfm9_original[x].astype('float').sum(axis=1)
        y=value[-1]
        if y == 'VEG0700':
            tmp2=dfm9_original[y].astype('float')*2
            dfm9_original[key]=tmp+tmp2
        else:
            print('NO SEAFOOD AND PLANT PROTEIN MISSING VEG0700, needs to be last in list')
    if key in ['hei_sodium']:
        x=value
        dfm4_original[key] = dfm4_original[x].astype('float')/1000
    if key in ['hei_addedsugars']:
        x=value
        dfm4_original[key] = dfm4_original[x].astype('float')*4
    if key in ['ripctsfa']:
        x=value[0]
        y=value[1]
        dfm4_original[key] = dfm4_original[x].astype('float')*dfm4_original[y].astype('float')
    if key in ['energy']:
        x=value
        dfm4_original[key] = dfm4_original[x].astype('float')/1000

hei_totveg
hei_greensbeans
hei_totfruit
hei_wholefruit
hei_wholegrains
hei_dairy
hei_totproteins
hei_seafoodplantprot
hei_sodium
hei_refinedgrains
hei_addedsugars
ripctsfa
energy
fats


In [192]:
dfm4_original['energy']

0      2.916060
1      3.098347
2      2.800854
3      3.118359
4      2.393655
5      2.956978
6      2.137675
7      1.721771
8      2.211933
9      1.893781
10     1.937645
11     3.412986
12     2.712041
13     2.771722
14     2.292539
15     2.281528
16     3.397806
17     1.982607
18     3.335774
19     3.062925
20     2.427762
21     3.410711
22     2.487761
23     2.473409
24     1.480256
25     3.514266
26     2.765407
27     2.967716
28     2.661781
29     1.867489
         ...   
330    1.869316
331    2.597143
332    1.383955
333    1.964563
334    2.433960
335    2.336014
336    1.792297
337    2.068107
338    2.676542
339    1.839390
340    1.722505
341    0.946801
342    1.948325
343    1.602056
344    1.448908
345    3.040873
346    1.120894
347    1.402019
348    1.988606
349    1.588981
350    1.965027
351    2.103980
352    1.265519
353    1.514757
354    1.579669
355    3.094807
356    2.770811
357    1.483241
358    2.575736
359    2.869768
Name: energy, Length: 36

# Important functions

In [ ]:
adeq_check<-function(inital, new, parameter){
  #parameters are based on the HEI 2015 from NDSR SAS HEI script
  v5<-c(data$hei_totveg, data$hei_greensbeans, data$hei_totfruit, data$hei_wholefruit, data$hei_totproteins, data$hei_seafoodplantprot)
  
  if(inital %in% v5 == TRUE){
    print("neat")
    init= inital/dfm4_original['Energy (kcal)']
    
    new	<-ifelse(init==0, 0, NA)
    new	<-ifelse(init >= parameter, 5, NA)
    new	<-ifelse(is.na(new), 5*(init/parameter),new)
    
  }
}



In [219]:
def adeq_check(inputt, output, parameter):
    print(inputt)
    if inputt in ['hei_totveg','hei_greensbeans', 'hei_totfruit', 'hei_wholefruit', 'hei_totproteins', 'hei_seafoodplantprot']:
        tmp = dfm9_original[inputt]/dfm4_original['energy']
        dfm9_original[output] = [5 if x >= parameter else 5*(x/parameter) for x in tmp]
    if inputt in ['hei_wholegrains', 'hei_dairy']:
        print('hi')
        tmp = dfm4_original[inputt]/dfm4_original['energy']
        dfm4_original[output] = [10 if x >= parameter else 10*(x/parameter) for x in tmp]
    if inputt in ['fats']:
        FARMIN=1.2
        FARMAX=2.5
        x=
        tmp = dfm4_original[inputt]/dfm4_original['energy']
        dfm4_original[output] = [10 if x >= parameter else 10*(x/parameter) for x in tmp] 
   





In [220]:
adeq_check('hei_wholegrains','test2',1.1)

hei_wholegrains
hi


In [221]:
dfm4_original['test2']

0      10.000000
1       0.000000
2       1.239882
3       0.000000
4       3.718163
5       4.399455
6       8.849887
7       4.334860
8      10.000000
9      10.000000
10      4.794949
11      6.922757
12     10.000000
13     10.000000
14      5.285922
15      3.984570
16      8.026570
17      5.617030
18     10.000000
19     10.000000
20     10.000000
21      3.851503
22      2.930711
23      8.317560
24     10.000000
25     10.000000
26      8.336764
27      5.397478
28     10.000000
29      9.083661
         ...    
330     0.000000
331     1.449145
332     0.000000
333    10.000000
334    10.000000
335     0.000000
336     2.946955
337     3.107805
338     2.462472
339     2.411867
340     9.114632
341     0.000000
342    10.000000
343     0.000000
344     0.000000
345    10.000000
346    10.000000
347     0.000000
348     7.232110
349     0.000000
350     2.891471
351     0.000000
352    10.000000
353     6.001563
354    10.000000
355     0.000000
356     1.181144
357     3.5609

### Find Columns with NaN

In [9]:
empty_df = dfm4_original[dfm4_original.isna().any(axis=1)]
for a in dfm4_original: 
       if dfm4_original[a].isna().any(axis=0) == True:
            #print(dfm4_original[a].head(30))
            del dfm4_original[a]

In [10]:
dfm4_original.empty
np.where(pd.isnull(dfm4_original))
np.where(dfm4_original.applymap(lambda x: x == ''))

(array([], dtype=int64), array([], dtype=int64))

## Select Relevant Rows:

In [46]:
temp_list = []            
for file in file_dict["set_04"]["mom"]["files"]:
    temp_df = pd.read_csv(file,encoding='latin1', sep="\t")
    print("Loading file {}...........Dataframe size: {}".format(file.split("/")[-1], temp_df.shape))
    updated_df = temp_df[["Participant ID", "Date of Intake", "Day of Intake", "Visit Number", "Site ID", "Total Grams", "Energy (kcal)", "Total Fat (g)", 
                  "Total Carbohydrate (g)", "Total Protein (g)", "Animal Protein (g)", "Vegetable Protein (g)", "Cholesterol (mg)", 
                  "Total Saturated Fatty Acids (SFA) (g)", "Total Monounsaturated Fatty Acids (MUFA) (g)", "Total Polyunsaturated Fatty Acids (PUFA) (g)", 
                  "Total Vitamin A Activity (International Units) (IU)", "Vitamin D (calciferol) (mcg)", "Galactose (g)", "Glucose (g)", "Lactose (g)", 
                  "Maltose (g)", "Sucrose (g)", "Total Dietary Fiber (g)", "Soluble Dietary Fiber (g)", "Insoluble Dietary Fiber (g)", "Total Folate (mcg)", 
                  "Vitamin B-12 (cobalamin) (mcg)", "Magnesium (mg)", "Iron (mg)", "Zinc (mg)", "Copper (mg)", "Selenium (mcg)", "PUFA 20:4 (arachidonic acid) (g)", 
                  "PUFA 20:5 (eicosapentaenoic acid [EPA]) (g)", "PUFA 22:6 (docosahexaenoic acid [DHA]) (g)", "Tryptophan (g)", "Threonine (g)", "Isoleucine (g)",
                  "Leucine (g)", "Lysine (g)", "Methionine (g)", "Cystine (g)", "Phenylalanine (g)", "Tyrosine (g)", "Valine (g)", "Arginine (g)", "Histidine (g)",
                  "Alanine (g)", "Aspartic Acid (g)", "Glutamic Acid (g)", "Glycine (g)", "Proline (g)", "Serine (g)", "Aspartame (mg)", "Saccharin (mg)", "Caffeine (mg)",
                  "Oxalic Acid (mg)", "Water (g)", "% Calories from Fat", "% Calories from Carbohydrate", "% Calories from Protein", "% Calories from Alcohol", 
                  "% Calories from SFA", "% Calories from MUFA", "% Calories from PUFA", "Polyunsaturated to Saturated Fat Ratio", "Cholesterol to Saturated Fatty Acid Index",
                  "Total Vitamin A Activity (Retinol Equivalents) (mcg)", "TRANS 18:1 (trans-octadecenoic acid) (g)", "TRANS 18:2 (trans-octadecadienoic acid) (g)", 
                  "TRANS 16:1 (trans-hexadecenoic acid) (g)", "Total Trans-Fatty Acids (TRANS) (g)", "Water (g)", "% Calories from Fat", "% Calories from Carbohydrate",
                  "% Calories from Protein", "% Calories from Alcohol", "% Calories from SFA", "% Calories from MUFA", "% Calories from PUFA", "Polyunsaturated to Saturated Fat Ratio",
                  "Cholesterol to Saturated Fatty Acid Index", "Total Vitamin A Activity (Retinol Equivalents) (mcg)", "TRANS 18:1 (trans-octadecenoic acid) (g)",
                  "TRANS 18:2 (trans-octadecadienoic acid) (g)", "TRANS 16:1 (trans-hexadecenoic acid) (g)", "Total Trans-Fatty Acids (TRANS) (g)", "Beta-Carotene (provitamin A carotenoid) (mcg)",
                  "Lutein + Zeaxanthin (mcg)", "Lycopene (mcg)", "Dietary Folate Equivalents (mcg)", "Natural Folate (food folate) (mcg)", "Synthetic Folate (folic acid) (mcg)",
                  "Niacin Equivalents (mg)", "Total Sugars (g)", "Omega-3 Fatty Acids (g)", "Manganese (mg)", "Vitamin E (International Units) (IU)", "Added Sugars (by Available Carbohydrate) (g)",
                  "Glycemic Index (glucose reference)", "Glycemic Load (glucose reference)", "Choline (mg)", "PUFA 18:3 n-3 (alpha-linolenic acid [ALA]) (g)"]]
    # Update ID to be consistent -- keeping only first 6 digits, no extra "_" tags
    for val in updated_df["Participant ID"]:
        _id = str(val).split("_")[0]
        #print(_id)
        updated_df.replace(val, _id, inplace=True)
    #print(updated_df["Participant ID"])
    temp_list.append(updated_df)
    #print(file)
    #temp_df =  pd.read_csv(file, sep="\t", encoding='latin1')
    #temp_list.append(temp_df)
    #print(temp_df.shape)
dfm4 = pd.concat(temp_list, ignore_index=True)
print("Final dataframe size: ", dfm4.shape)
dfm4.head()

Loading file UNCMomJuly04.txt...........Dataframe size: (26, 226)
Loading file UMNMomJuly04.txt...........Dataframe size: (122, 226)
Loading file MomOct1804.txt...........Dataframe size: (29, 226)
Loading file UMNUNCMom04.txt...........Dataframe size: (17, 226)
Loading file BCPMomAP04.txt...........Dataframe size: (27, 226)
Loading file BCPJulAugMom04.txt...........Dataframe size: (35, 226)
Final dataframe size:  (256, 104)


,Participant ID,Date of Intake,Day of Intake,Visit Number,Site ID,Total Grams,Energy (kcal),Total Fat (g),Total Carbohydrate (g),Total Protein (g),...,Niacin Equivalents (mg),Total Sugars (g),Omega-3 Fatty Acids (g),Manganese (mg),Vitamin E (International Units) (IU),Added Sugars (by Available Carbohydrate) (g),Glycemic Index (glucose reference),Glycemic Load (glucose reference),Choline (mg),PUFA 18:3 n-3 (alpha-linolenic acid [ALA]) (g)
0,011370,05/30/2017,2,2,UNC,2872.041,3098.347,155.139,342.807,100.001,...,51.546,134.092,4.121,4.157,18.835,59.016,58.504,181.208,293.112,4.086
1,011370,08/23/2017,3,1,UNC,2743.424,2800.854,105.956,358.233,115.060,...,52.776,185.565,3.582,3.603,14.619,97.642,54.701,183.927,499.411,3.486
2,017541,05/30/2017,2,1,UNC,2790.541,3118.359,156.893,341.159,101.941,...,53.032,126.078,4.358,4.038,21.658,50.419,58.746,181.272,290.828,4.324
3,043358,09/21/2017,4,1,UNC,1964.220,1937.645,77.584,233.575,82.651,...,40.912,112.576,0.938,2.525,14.065,84.990,58.354,125.826,629.616,0.775
4,107008,06/15/2017,4,1,UNC,5106.196,1982.607,97.905,229.872,74.010,...,41.231,117.427,1.224,5.262,31.164,25.129,50.219,97.498,310.720,1.211


In [47]:
dfm4 = dfm4.sort_values(by="Participant ID")
dfm4.head()

,Participant ID,Date of Intake,Day of Intake,Visit Number,Site ID,Total Grams,Energy (kcal),Total Fat (g),Total Carbohydrate (g),Total Protein (g),...,Niacin Equivalents (mg),Total Sugars (g),Omega-3 Fatty Acids (g),Manganese (mg),Vitamin E (International Units) (IU),Added Sugars (by Available Carbohydrate) (g),Glycemic Index (glucose reference),Glycemic Load (glucose reference),Choline (mg),PUFA 18:3 n-3 (alpha-linolenic acid [ALA]) (g)
177,001999,09/18/2018,2,1,UMN,3557.076,2194.330,78.515,306.670,64.076,...,32.159,112.201,1.171,5.618,11.167,47.911,51.077,139.662,223.584,1.170
0,011370,05/30/2017,2,2,UNC,2872.041,3098.347,155.139,342.807,100.001,...,51.546,134.092,4.121,4.157,18.835,59.016,58.504,181.208,293.112,4.086
1,011370,08/23/2017,3,1,UNC,2743.424,2800.854,105.956,358.233,115.060,...,52.776,185.565,3.582,3.603,14.619,97.642,54.701,183.927,499.411,3.486
2,017541,05/30/2017,2,1,UNC,2790.541,3118.359,156.893,341.159,101.941,...,53.032,126.078,4.358,4.038,21.658,50.419,58.746,181.272,290.828,4.324
178,020784,09/25/2018,2,5,UMN,1524.126,1096.180,50.882,104.764,63.370,...,31.886,56.387,2.862,2.297,11.055,37.746,54.179,49.180,297.010,0.529


In [48]:
# add mom or child identifier column
dfm4["Mom(0) or Child(1) ID"] = 0
print(dfm4.shape)
dfm4["Mom(0) or Child(1) ID"].head()

(256, 105)


177    0
0      0
1      0
2      0
178    0
Name: Mom(0) or Child(1) ID, dtype: int64

In [49]:
dfm4['Sphingomyelin (mg)'] = dfm4[['Choline (mg)']].div(4.366)
dfm4['Sphingomyelin (mg)'].head()

177     51.210261
0       67.135135
1      114.386395
2       66.612002
178     68.027943
Name: Sphingomyelin (mg), dtype: float64

In [50]:
dfm4['Phosphatidycholine (mg)'] = dfm4[['Choline (mg)']].multiply(0.479)
dfm4['Phosphatidycholine (mg)'].head()

177    107.096736
0      140.400648
1      239.217869
2      139.306612
178    142.267790
Name: Phosphatidycholine (mg), dtype: float64

In [51]:
avg_df=dfm4.groupby(["Participant ID", "Visit Number"]).mean(numeric_only=True)
#reset index
avg_df.reset_index(inplace=True)  
avg_df.head()

,Participant ID,Visit Number,Day of Intake,Total Grams,Energy (kcal),Total Fat (g),Total Carbohydrate (g),Total Protein (g),Animal Protein (g),Vegetable Protein (g),...,Manganese (mg),Vitamin E (International Units) (IU),Added Sugars (by Available Carbohydrate) (g),Glycemic Index (glucose reference),Glycemic Load (glucose reference),Choline (mg),PUFA 18:3 n-3 (alpha-linolenic acid [ALA]) (g),Mom(0) or Child(1) ID,Sphingomyelin (mg),Phosphatidycholine (mg)
0,001999,1,2.0,3557.076,2194.330,78.515,306.670,64.076,23.277,40.800,...,5.618,11.167,47.911,51.077,139.662,223.584,1.170,0.0,51.210261,107.096736
1,011370,1,3.0,2743.424,2800.854,105.956,358.233,115.060,88.510,26.550,...,3.603,14.619,97.642,54.701,183.927,499.411,3.486,0.0,114.386395,239.217869
2,011370,2,2.0,2872.041,3098.347,155.139,342.807,100.001,47.819,52.182,...,4.157,18.835,59.016,58.504,181.208,293.112,4.086,0.0,67.135135,140.400648
3,017541,1,2.0,2790.541,3118.359,156.893,341.159,101.941,48.123,53.818,...,4.038,21.658,50.419,58.746,181.272,290.828,4.324,0.0,66.612002,139.306612
4,020784,5,2.0,1524.126,1096.180,50.882,104.764,63.370,47.624,15.748,...,2.297,11.055,37.746,54.179,49.180,297.010,0.529,0.0,68.027943,142.267790


In [70]:
concat_filename = "Mom_BCP_dataset4"
csv_filename = "{}.csv".format(concat_filename)
tsv_filename = "{}.tsv".format(concat_filename)
xlsx_filename = "{}.xlsx".format(concat_filename)
concat_filepath = os.path.join(csv_filename)
xlsx_filepath = os.path.join(xlsx_filename)
tsv_filepath = os.path.join(tsv_filename)

#print( new_filepath)
# write concatentation to file
writer = pd.ExcelWriter(xlsx_filepath, engine='xlsxwriter')
avg_df.to_excel(writer, sheet_name='Sheet1', index=False)
#concat_df.to_excel(concat_filepath)
avg_df.to_csv(concat_filepath, index=False, sep="\t", header=True)
avg_df.to_csv(tsv_filepath, index=False, sep="\t", header=True)
print("Written concat file, ", concat_filepath)
#umn_df = final_df[concat_df['Site ID'] == "UMN"]
#umn_df.head()

Written concat file,  Mom_BCP_dataset4.csv


## Inspect Data further:

In [55]:
avg_df["Participant ID"].unique()
avg_df["Day of Intake"].unique()
avg_df.columns.values


array(['Participant ID', 'Visit Number', 'Day of Intake', 'Total Grams',
       'Energy (kcal)', 'Total Fat (g)', 'Total Carbohydrate (g)',
       'Total Protein (g)', 'Animal Protein (g)', 'Vegetable Protein (g)',
       'Cholesterol (mg)', 'Total Saturated Fatty Acids (SFA) (g)',
       'Total Monounsaturated Fatty Acids (MUFA) (g)',
       'Total Polyunsaturated Fatty Acids (PUFA) (g)',
       'Total Vitamin A Activity (International Units) (IU)',
       'Vitamin D (calciferol) (mcg)', 'Galactose (g)', 'Glucose (g)',
       'Lactose (g)', 'Maltose (g)', 'Sucrose (g)',
       'Total Dietary Fiber (g)', 'Soluble Dietary Fiber (g)',
       'Insoluble Dietary Fiber (g)', 'Total Folate (mcg)',
       'Vitamin B-12 (cobalamin) (mcg)', 'Magnesium (mg)', 'Iron (mg)',
       'Zinc (mg)', 'Copper (mg)', 'Selenium (mcg)',
       'PUFA 20:4 (arachidonic acid) (g)',
       'PUFA 20:5 (eicosapentaenoic acid [EPA]) (g)',
       'PUFA 22:6 (docosahexaenoic acid [DHA]) (g)', 'Tryptophan (g)',
      